In [17]:
'''
'FAVC'      고칼로리 음식자주 섭취   예/아니오 고칼로리 음식을 섭취하는지 여부
'FCVC'      채소 섭취 빈도   예/아니오
'NCP'      주요 식사 횟수
'CAEC'    식사 사이의 음식 섭취
'CH2O'    물섭취량
'''

"\n'FAVC'      고칼로리 음식자주 섭취   예/아니오 고칼로리 음식을 섭취하는지 여부\n'FCVC'      채소 섭취 빈도   예/아니오\n'NCP'      주요 식사 횟수\n'CAEC'    식사 사이의 음식 섭취\n'CH2O'    물섭취량\n"

In [18]:
### 데이터 파일 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import koreanize_matplotlib

file_name = 'ObesityDataSet_rev.csv'
Obesity_df = pd.read_csv(file_name)
Obesity_df

,FAVC,FCVC,NCP,CAEC,CH2O,NObeyesdad
0,no,2.0,3.0,Sometimes,2.000000,Normal_Weight
1,no,3.0,3.0,Sometimes,3.000000,Normal_Weight
2,no,2.0,3.0,Sometimes,2.000000,Normal_Weight
3,no,3.0,3.0,Sometimes,2.000000,Overweight_Level_I
4,no,2.0,1.0,Sometimes,2.000000,Overweight_Level_II
...,...,...,...,...,...,...
2106,yes,3.0,3.0,Sometimes,1.728139,Obesity_Type_III
2107,yes,3.0,3.0,Sometimes,2.005130,Obesity_Type_III
2108,yes,3.0,3.0,Sometimes,2.054193,Obesity_Type_III
2109,yes,3.0,3.0,Sometimes,2.852339,Obesity_Type_III


## 데이터 전처리

In [19]:
## NObeyesdad 열값 0,1로 바꾸기
num = Obesity_df.shape[0]
Obesity_df['Obesity'] = 0

for i in range(num):
    if Obesity_df.iloc[i]['NObeyesdad'] == 'Normal_Weight':
        Obesity_df.loc[i,'Obesity'] = 'Normal'
    elif Obesity_df.iloc[i]['NObeyesdad'] == 'Insufficient_Weight':
        Obesity_df.loc[i,'Obesity'] = 'Normal'
    elif Obesity_df.iloc[i]['NObeyesdad'] == 'Overweight_Level_I':
        Obesity_df.loc[i,'Obesity'] = 'Normal'
    elif Obesity_df.iloc[i]['NObeyesdad'] == 'Overweight_Level_II':
        Obesity_df.loc[i,'Obesity'] = 'Normal'
    else:
        Obesity_df.loc[i,'Obesity'] = 'Obesity'
Obesity_df


C:\Users\KDP-14\AppData\Local\Temp\ipykernel_18252\714365054.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Normal' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Obesity_df.loc[i,'Obesity'] = 'Normal'


,FAVC,FCVC,NCP,CAEC,CH2O,NObeyesdad,Obesity
0,no,2.0,3.0,Sometimes,2.000000,Normal_Weight,Normal
1,no,3.0,3.0,Sometimes,3.000000,Normal_Weight,Normal
2,no,2.0,3.0,Sometimes,2.000000,Normal_Weight,Normal
3,no,3.0,3.0,Sometimes,2.000000,Overweight_Level_I,Normal
4,no,2.0,1.0,Sometimes,2.000000,Overweight_Level_II,Normal
...,...,...,...,...,...,...,...
2106,yes,3.0,3.0,Sometimes,1.728139,Obesity_Type_III,Obesity
2107,yes,3.0,3.0,Sometimes,2.005130,Obesity_Type_III,Obesity
2108,yes,3.0,3.0,Sometimes,2.054193,Obesity_Type_III,Obesity
2109,yes,3.0,3.0,Sometimes,2.852339,Obesity_Type_III,Obesity


In [20]:
## 'CAEC','NObeyesdad' drop하기
Obesity_df.drop(['CAEC','NObeyesdad'],axis=1,inplace=True)

# 'FAVC' 열값 0,1로 바꾸기
for i in range(num):
    if Obesity_df.iloc[i]['FAVC'] == 'no':
        Obesity_df.loc[i,'FAVC'] = 0
    elif Obesity_df.iloc[i]['FAVC'] == 'yes':
        Obesity_df.loc[i,'FAVC'] = 1


Obesity_df['FAVC']=Obesity_df['FAVC'].astype(int)
Obesity_df['FCVC']=Obesity_df['FCVC'].astype(int)
Obesity_df['NCP']=Obesity_df['NCP'].astype(int)
Obesity_df['CH2O']=Obesity_df['CH2O'].astype(int)
## 데이터 프레임 정보 확인

Obesity_df.info()
# #   Column   Non-Null Count  Dtype
# ---  ------   --------------  -----
#  0   FAVC     2111 non-null   int32
#  1   FCVC     2111 non-null   int32
#  2   NCP      2111 non-null   int32
#  3   CH2O     2111 non-null   int32
#  4   Obesity  2111 non-null   int64 

# Obesity_df.set_index('Obesity',inplace=True)
# Obesity_df.sort_index(inplace=True)
Obesity_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   FAVC     2111 non-null   int32 
 1   FCVC     2111 non-null   int32 
 2   NCP      2111 non-null   int32 
 3   CH2O     2111 non-null   int32 
 4   Obesity  2111 non-null   object
dtypes: int32(4), object(1)
memory usage: 49.6+ KB


,FAVC,FCVC,NCP,CH2O,Obesity
0,0,2,3,2,Normal
1,0,3,3,3,Normal
2,0,2,3,2,Normal
3,0,3,3,2,Normal
4,0,2,1,2,Normal
...,...,...,...,...,...
2106,1,3,3,1,Obesity
2107,1,3,3,2,Obesity
2108,1,3,3,2,Obesity
2109,1,3,3,2,Obesity


In [22]:

# 독립 변수(X)와 종속 변수(y) 설정
X = Obesity_df['FAVC'].values.reshape(-1, 1)
y = list(Obesity_df['Obesity'])

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 생성 및 학습
model = LogisticRegression()
result=model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("분류 보고서:\n", class_report)


정확도: 0.5741324921135647
분류 보고서:
               precision    recall  f1-score   support

      Normal       0.95      0.23      0.37       346
     Obesity       0.52      0.99      0.68       288

    accuracy                           0.57       634
   macro avg       0.73      0.61      0.52       634
weighted avg       0.75      0.57      0.51       634

